In [1]:
import cv2
import torch
import numpy as np
from facenet_pytorch import MTCNN

In [12]:
import torch.nn as nn

In [13]:
class HGRecognition(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = self.conv_block(1, 32, pool=False)
        self.conv2 = self.conv_block(32, 32, pool=True)
        self.conv3 = self.conv_block(32, 64, pool=True)
        self.drop = nn.Dropout(0.3)
        self.dense = nn.Sequential(nn.Flatten(), nn.Linear(64*6*6, 256), nn.ReLU(inplace=True), 
                                  nn.Linear(256, 10), nn.Softmax(dim=1))
        
    @staticmethod
    def conv_block(in_feat, out_feat, pool=False):
        layers = [nn.Conv2d(in_feat, out_feat, kernel_size=3), nn.ReLU(inplace=True)]
        if pool:
            layers.append(nn.MaxPool2d(kernel_size=3))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.drop(out)
        out = self.conv3(out)
        out = self.drop(out)
        #print(out.size())
        out = self.dense(out)
        
        return out

In [37]:
class FaceAndHandDetector():
    
    def __init__(self, mtcnn, hg_model):
        self.mtcnn = mtcnn
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.hg_model = hg_model.to(self.device)
        self.hg_model.load_state_dict(torch.load('./Models/Hand_Gesture_Recognition.pth'))
        self.hg_model.eval()
        
        
    def _draw(self, frame, boxes, probs, landmarks):
        try:
            for box, prob, ld in zip(boxes, probs, landmarks):
                # Рисуем обрамляющий прямоугольник лица на кадре
                cv2.rectangle(frame,
                              (box[0], box[1]),
                              (box[2], box[3]),
                              (0, 0, 255),
                              thickness=2)

                # Рисуем особенные точки
                cv2.circle(frame, tuple(ld[0]), 5, (0, 0, 255), -1)
                cv2.circle(frame, tuple(ld[1]), 5, (0, 0, 255), -1)
                cv2.circle(frame, tuple(ld[2]), 5, (0, 0, 255), -1)
                cv2.circle(frame, tuple(ld[3]), 5, (0, 0, 255), -1)
                cv2.circle(frame, tuple(ld[4]), 5, (0, 0, 255), -1)
        except:
            print('Something wrong in draw function!')

        return frame
    
    @staticmethod
    def digit_to_classname(digit):
        cats = ["palm", 'l','fist','fist_moved','thumb','index','ok','palm_moved','c','down']
        return cats[digit]
    
    def run(self):
        cap = cv2.VideoCapture(0)
        
        while True:
            ret, frame = cap.read()
            
            try:
                # детектируем расположение лица на кадре, вероятности на сколько это лицо
                # и особенные точки лица
                boxes, probs, landmarks = self.mtcnn.detect(frame, landmarks=True)
                if isinstance(boxes, np.ndarray):
                    # Рисуем на кадре рамку для лица, если на кадре лицо.
                    self._draw(frame, boxes, probs, landmarks)
                else:
                    #если на кадре не лицо, пытаемся определить жест.
                    hand = cv2.resize(frame, (64, 64))
                    hand = cv2.cvtColor(hand, cv2.COLOR_BGR2GRAY)
                    torch_hand = torch.from_numpy(hand).unsqueeze(0).to(self.device).float()
                    gesture_probs = self.hg_model(torch_hand[None, ...])
                    gesture = self.digit_to_classname(gesture_probs.argmax())
                    h, w = frame.shape[:2]
                    cv2.putText(frame, gesture, (w//2, h), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    
            except Exception as e:
                print(f'Something wrong in main cycle! {e}')

            # Показываем кадр в окне, и назвываем его(окно) - 'Face Detection'
            cv2.imshow('Face Detection', frame)
            
            # Функция, которая проверяет нажатие на клавишу 'q'
            # Если нажатие произошло - выход из цикла. Конец работы приложения
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
        # Очищаем все объекты opencv, что мы создали
        cap.release()
        cv2.destroyAllWindows()

In [38]:
mtcnn = MTCNN()
hg_model = HGRecognition()
fd = FaceAndHandDetector(mtcnn, hg_model)
fd.run()

<ipython-input-37-3fad3ffda3ea>:15: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.rectangle(frame,
<ipython-input-37-3fad3ffda3ea>:22: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.circle(frame, tuple(ld[0]), 5, (0, 0, 255), -1)
<ipython-input-37-3fad3ffda3ea>:23: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  cv2.circle(frame, tuple(ld[1]), 5, (0, 0, 255), -1)
<ipython-input-37-3fad3ffda3ea>:24: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
 